# 如何传输图表的完整状态

LangGraph 支持多种流模式。主要有：

- `values`：此流模式流回图表的值。这是
**调用每个节点后图的完整状态**。
- `updates`：此流模式将更新流回图表。这是
**在调用每个节点后更新图的状态**。

本指南涵盖 `streamMode="values"`。

In [1]:
// process.env.OPENAI_API_KEY = "sk-...";

## 定义状态

状态是图中所有节点的接口。


In [2]:
import { Annotation } from "@langchain/langgraph";
import { BaseMessage } from "@langchain/core/messages";

const StateAnnotation = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
});

## 设置工具

我们将首先定义我们想要使用的工具。对于这个简单的例子，我们将
使用创建占位符搜索引擎。然而，创建起来确实很容易
您自己的工具 - 请参阅文档
[此处](https://js.langchain.com/docs/how_to/custom_tools) 了解如何操作
那。


In [3]:
import { tool } from "@langchain/core/tools";
import { z } from "zod";

const searchTool = tool(async ({ query: _query }: { query: string }) => {
  // 这是实际实现的占位符
  return "Cold, with a low of 3℃";
}, {
  name: "search",
  description:
    "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
  schema: z.object({
    query: z.string().describe("The query to use in your search."),
  }),
});

await searchTool.invoke({ query: "What's the weather like?" });

const tools = [searchTool];

我们现在可以将这些工具包装在一个简单的
[ToolNode](/langgraphjs/reference/classes/langgraph_prebuilt.ToolNode.html)。
每当工具（函数）被调用时，该对象就会实际运行它们
我们的法学硕士。


In [4]:
import { ToolNode } from "@langchain/langgraph/prebuilt";

const toolNode = new ToolNode(tools);

## 设置模型

现在我们将加载
[聊天模型](https://js.langchain.com/docs/concepts/chat_models/)。

1.它应该与消息一起使用。我们将以表格形式代表所有代理状态
消息，因此它需要能够与它们很好地配合。
2.它应该与
[工具调用](https://js.langchain.com/docs/how_to/tool_calling/#passing-tools-to-llms),
这意味着它可以在响应中返回函数参数。

<div class="警告提示">
<p class="admonition-title">注意</p>
<p>
这些模型要求不是使用 LangGraph 的一般要求 - 它们只是这个示例的要求。
</p>
</div>

In [5]:
import { ChatOpenAI } from "@langchain/openai";

const model = new ChatOpenAI({ model: "gpt-4o" });

完成此操作后，我们应该确保模型知道它具有这些
可供调用的工具。我们可以通过调用来做到这一点
[bindTools](https://v01.api.js.langchain.com/classes/langchain_core_language_models_chat_models.BaseChatModel.html#bindTools)。

In [6]:
const boundModel = model.bindTools(tools);

## 定义图表

我们现在可以把它们放在一起。

In [7]:
import { END, START, StateGraph } from "@langchain/langgraph";
import { AIMessage } from "@langchain/core/messages";

const routeMessage = (state: typeof StateAnnotation.State) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1] as AIMessage;
  // 如果没有调用任何工具，我们就可以完成（响应用户）
  if (!lastMessage?.tool_calls?.length) {
    return END;
  }
  // 否则，如果有，我们继续并调用工具
  return "tools";
};

const callModel = async (
  state: typeof StateAnnotation.State,
) => {
  // 对于 @langchain/core < 0.2.3 的版本，您必须调用 `.stream()`
  // 并从块聚合消息而不是调用“.invoke()”。
  const { messages } = state;
  const responseMessage = await boundModel.invoke(messages);
  return { messages: [responseMessage] };
};

const workflow = new StateGraph(StateAnnotation)
  .addNode("agent", callModel)
  .addNode("tools", toolNode)
  .addEdge(START, "agent")
  .addConditionalEdges("agent", routeMessage)
  .addEdge("tools", "agent");

const graph = workflow.compile();

## 流值

我们现在可以与代理进行交互。在交互之间你可以获取和更新
状态。


In [8]:
let inputs = { messages: [{ role: "user", content: "what's the weather in sf" }] };

for await (
  const chunk of await graph.stream(inputs, {
    streamMode: "values",
  })
) {
  console.log(chunk["messages"]);
  console.log("\n====\n");
}

[ [ 'user', "what's the weather in sf" ] ]

====

[
  [ 'user', "what's the weather in sf" ],
  AIMessage {
    "id": "chatcmpl-9y660d49eLzT7DZeBk2ZmX8C5f0LU",
    "content": "",
    "additional_kwargs": {
      "tool_calls": [
        {
          "id": "call_iD5Wk4vPsTckffDKJpEQaMkg",
          "type": "function",
          "function": "[Object]"
        }
      ]
    },
    "response_metadata": {
      "tokenUsage": {
        "completionTokens": 17,
        "promptTokens": 70,
        "totalTokens": 87
      },
      "finish_reason": "tool_calls",
      "system_fingerprint": "fp_3aa7262c27"
    },
    "tool_calls": [
      {
        "name": "search",
        "args": {
          "query": "current weather in San Francisco"
        },
        "type": "tool_call",
        "id": "call_iD5Wk4vPsTckffDKJpEQaMkg"
      }
    ],
    "invalid_tool_calls": [],
    "usage_metadata": {
      "input_tokens": 70,
      "output_tokens": 17,
      "total_tokens": 87
    }
  }
]

====

[
  [ 'user', "